In [1]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Dropout, Input, Concatenate
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer
import nltk
nltk.download('punkt')
import tensorflow as tf

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
import pandas as pd
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, GRU, Dense, Embedding, Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

from keras.layers import Layer
import keras.backend as K
from tensorflow.keras import regularizers

In [4]:
import pandas as pd
train = pd.read_csv('/content/drive/My Drive/Hackathon/Sentiment Analysis/train_2kmZucJ.csv')
test = pd.read_csv('/content/drive/My Drive/Hackathon/Sentiment Analysis/test_oJQbWVk.csv')

In [5]:
import re
tweet = pd.concat( [train['tweet'], test['tweet']], axis = 0, ignore_index=True)
pattern = re.compile( r'(www|http|https)[-%+:/_.=?&\w]*' )
pattern2 = re.compile( r'(#\w+|\w+)\.(\w+)[-%+:/_.=?&\w]*' )
pattern3 = re.compile( r'[^A-Za-z0-9 ]+' )
pattern4 = re.compile( r'\b[Aa]{1,}[rR]{1,}[gGhH]{3,}\b')
pattern5 = re.compile( r'[0-9]+' )
pattern6 = re.compile( r'\b[Aa]{1,}[hH]{2,}\b' )
pattern7 = re.compile(r'\b[gG]{1,}[rR]{3,}\b')
pattern8 = re.compile(r'\b[uU]+[gG]+[hH]+\b')

pattern9 = re.compile( r'(\bh{3,}\b|\Bh{3,}\B|\bh{3,}\b|\bh{3,}\B|\Bh{3,}\b)' )
pattern10 = re.compile( r'(\bo{3,}\b|\Bo{3,}\B|\bo{3,}\b|\bo{3,}\B|\Bo{3,}\b)' )

pattern11 = re.compile( r'\bgr8\b' )
pattern12 = re.compile( r'\bgd\b' )
pattern13 = re.compile( r'\b[oO]+[mM]+[fF]*[gG]+\b' )
pattern14 = re.compile( r'\b(lol|lmao)\b' )
pattern15 = re.compile( r'(\ba{3,}\b|\Ba{3,}\B|\ba{3,}\b|\ba{3,}\B|\Ba{3,}\b)' )
pattern16 = re.compile( r'(\w)\1{3,}' )
pattern17 = re.compile( r'\b(?:haha)+[ah]*\b' )
pattern18 = re.compile( r'\b(thx|thanx|tq)\b' )

for i in range( tweet.shape[0] ):
  pat = re.sub( pattern,'',tweet[i].lower() )
  pat = re.sub( pattern2,'',pat )
  pat = re.sub( pattern3,'',pat )
  pat = re.sub( pattern4,'', pat )
  pat = re.sub( pattern11, 'great', pat )
  pat = re.sub( pattern5, '', pat )
  pat = re.sub( pattern6, '', pat )
  pat = re.sub( pattern7, '', pat )
  pat = re.sub( pattern8, '', pat )
  pat = re.sub( pattern9, 'h', pat )
  pat = re.sub( pattern10, 'o', pat )
  pat = re.sub( pattern12, 'good', pat )
  pat = re.sub( pattern13, 'Oh my God', pat )
  pat = re.sub( pattern14, 'laughing', pat )
  pat = re.sub( pattern15, 'a', pat )
  pat = re.sub( pattern16, r'\1', pat )
  pat = re.sub( pattern17, 'laughing', pat )
  pat = re.sub( pattern18, 'thanks', pat )
  pat = ' '.join( pat.split() )
  tweet[i] = pat

In [6]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(tweet)
vocab_length = len(word_tokenizer.word_index) + 1
vocab_length

18813

In [7]:
embedded_sentences = word_tokenizer.texts_to_sequences(tweet)
print(embedded_sentences)

[[1798, 6977, 1495, 32, 98, 71, 43, 149, 121, 145, 135, 1], [93, 8, 6978, 3551, 59, 105, 6, 3, 2524, 261, 17, 300, 40, 6979], [88, 26, 22, 115, 13, 143, 936, 6980, 4527, 537, 1, 150, 376, 1309], [46, 2219, 4, 168, 46, 6981, 4, 119, 255, 42, 306, 1, 43, 6982, 129], [101, 91, 340, 2, 196, 184, 936, 6, 12, 116, 8, 2220, 4, 28, 2525, 4, 495, 188, 11, 251, 228, 435], [1, 538, 102, 830, 78, 3, 15, 243, 69, 228, 141], [63, 11, 236, 295, 223, 236, 17, 300, 668], [10, 1101, 562, 172, 594, 595, 3552, 278, 1988, 10, 193, 4528, 2221, 4529, 1496, 6983, 6984, 2937, 6985, 392, 1218, 4530, 189, 787, 81], [2526, 6, 143, 237, 185, 1102, 6, 56, 1, 2527, 56, 801, 1989], [44, 58, 104, 1799, 552, 17, 136, 151, 276, 65, 244, 43, 2938], [266, 2, 73, 13, 90, 8, 10, 84, 97, 90, 14, 8, 10, 563, 128, 2528, 2939, 90, 14, 47, 1310, 139, 1639, 2222, 6986], [1219, 47, 2940, 6987, 67, 14, 286, 101, 4, 169, 14, 6, 2, 192, 937, 5, 1401, 23, 42, 360, 6988], [4531, 669, 19, 6, 5, 1, 4532, 2529, 197, 6989, 117, 77, 10, 253

In [8]:
from nltk.tokenize import word_tokenize

word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(tweet, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

padded_sentences = pad_sequences(embedded_sentences, length_long_sentence, padding='post')
print(padded_sentences)

[[ 1798  6977  1495 ...     0     0     0]
 [   93     8  6978 ...     0     0     0]
 [   88    26    22 ...     0     0     0]
 ...
 [  233   742     8 ...     0     0     0]
 [18812   112     4 ...     0     0     0]
 [   93     4    55 ...     0     0     0]]


In [9]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()
glove_file = open('/content/drive/My Drive/NLP/glove.6B.100d.txt', encoding="utf8")

In [10]:
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [11]:
embedding_matrix = zeros((vocab_length, 100))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [13]:
train_padded_sentences = padded_sentences[0:7920]
test_padded_sentences = padded_sentences[7920:]
train_sentiment = train['label']

In [14]:
def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [ ]:
inputs=Input((length_long_sentence,))
x=Embedding(input_dim = vocab_length,output_dim=100, weights=[embedding_matrix], input_length=length_long_sentence, trainable=True )(inputs)

att_in=Bidirectional( LSTM(100,return_sequences=True,dropout=0.8,recurrent_dropout=0.8) )(x)
att_out=attention()(att_in)
outputs=Dense(1,activation='sigmoid',trainable=True)(att_out)
model=Model(inputs,outputs)
#model.summary()

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(x=train_padded_sentences,y=train_sentiment,batch_size=64,epochs=20,verbose=1,shuffle=True,validation_split=0.2)

In [16]:
result = model.predict(test_padded_sentences)

In [17]:
result[ result < 0.5 ] = 0
result[ result >= 0.5 ] = 1
submission = pd.read_csv('/content/drive/MyDrive/Hackathon/Sentiment Analysis/sample_submission_LnhVWA4.csv')
submission.head()
submission['label'] = result
submission.to_csv('result.csv', index=False)